<a href="https://colab.research.google.com/github/TollanBerhanu/Semantic-search-on-Slack/blob/main/slack_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the dataset

In [3]:
import os
import pandas as pd

In [38]:
slack_data = '/content/drive/MyDrive/Colab Notebooks/dataset/slack-data/'
# cwd = os.path.join (os.getcwd(), slack_data)  # join with current_working_directory just in case

df = pd.read_json(slack_data + 'channels.json')

# channel_ids = [id for id in df['id']]
channel_names = [ name for name in df['name']]

channel_names
#channel_info --> (key: channel_id , value: channel_name)

['random', 'general', 'test']

In [39]:
# Helper functions
def lookup_channel(channel_id):
  channel_name = ''
  return channel_name

In [53]:
# import necessary libraries
import glob
import json

def extract_channel_data(channel_name):
  # use glob to get all the json files in the folder
  daily_json_files = glob.glob(slack_data + channel_name +'/*.json')

  # just return if the channel doesn't exist (or hasn't been exported yet)
  if not daily_json_files:
    return

  metadata = pd.DataFrame(columns = ['message', 'channel', 'date', 'time', 'user_id', 'user_name'])

  # loop over the list of json files (each json file includes every post in that channel for a single day)
  for f in daily_json_files:
    # read the json file
    # today_data = pd.read_json(f)
    with open(f, 'r') as file:
        # Read the contents
        data = file.read()
        # Parse the JSON data
        today_data = json.loads(data)

    today_date = f.split("/")[-1]  # 'f' is the full file path and file name
    print('Extracting...', today_date, type(today_data)) # the file name is the date

    # iterate through all the messages of the day
    for msg_data in today_data:
      print(msg_data)
      # Skip if its a "channel_join" type message or if the actual message content is empty
      if ('subtype' in msg_data) or (msg_data['text'] == ""):
        print('subtype' in msg_data, msg_data['text'] == "")
        continue
        # We should also remove any links, stickers, and other junk
        # We should replace @Member references by their actual names

      metadata.append ( {
            'message': today_data['text'],
            'channel': channel_name,
            'date': today_date,
            'time': today_data['ts'],
            'user_id': today_data['user'],
            'user_name': today_data['user_profile'] # We can also use 'real_name' if we wanted the full name of the user
      }, ignore_index = True )

  return metadata

In [54]:
extract_channel_data('general')

Extracting... 2023-06-19.json <class 'list'>
{'type': 'message', 'subtype': 'channel_join', 'ts': '1687165577.272839', 'user': 'U05DHDPL4FK', 'text': '<@U05DHDPL4FK> has joined the channel'}
True False
{'client_msg_id': '66b831cc-75c5-4a33-942f-3441bf83df6f', 'type': 'message', 'text': 'hello', 'user': 'U05DHDPL4FK', 'ts': '1687166197.580079', 'blocks': [{'type': 'rich_text', 'block_id': 'tSH', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'hello'}]}]}], 'team': 'T05D4QLFERY', 'user_team': 'T05D4QLFERY', 'source_team': 'T05D4QLFERY', 'user_profile': {'avatar_hash': 'g741f2dae79d', 'image_72': 'https://secure.gravatar.com/avatar/741f2dae79d044ac787e96f005d9e415.jpg?s=72&d=https%3A%2F%2Fa.slack-edge.com%2Fdf10d%2Fimg%2Favatars%2Fava_0025-72.png', 'first_name': 'kenenisaalemayhu0', 'real_name': 'kenenisaalemayhu0', 'display_name': '', 'team': 'T05D4QLFERY', 'name': 'kenenisaalemayhu0', 'is_restricted': False, 'is_ultra_restricted': False}}


TypeError: ignored

In [50]:
# create an Empty DataFrame
# object With column names only
df = pd.DataFrame(columns = ['Name', 'Articles', 'Improved'])
print(df)

# append rows to an empty DataFrame
df = df.append({'Name' : 'Ankit', 'Articles' : 97, 'Improved' : 2200},
        ignore_index = True)

df = df.append({'Name' : 'Aishwary', 'Articles' : 30, 'Improved' : 50},
        ignore_index = True)

df = df.append({'Name' : 'yash', 'Articles' : 17, 'Improved' : 220},
      ignore_index = True)

df

Empty DataFrame
Columns: [Name, Articles, Improved]
Index: []


<ipython-input-50-903e8d76ac97>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name' : 'Ankit', 'Articles' : 97, 'Improved' : 2200},
<ipython-input-50-903e8d76ac97>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name' : 'Aishwary', 'Articles' : 30, 'Improved' : 50},
<ipython-input-50-903e8d76ac97>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name' : 'yash', 'Articles' : 17, 'Improved' : 220},


,Name,Articles,Improved
0,Ankit,97,2200
1,Aishwary,30,50
2,yash,17,220


In [ ]:
# Cast the values of the column 'Content' into strings
df['Content'] = df['Content'].astype(str)

# Join the string values of all the rows in 'Content' into one large corpus of text
conversations = ' '.join(df['Content'])

Embedding

Generating embedding using sentence transformers

In [ ]:
!pip install --upgrade langchain  -q
!pip install sentence_transformers > /dev/null

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# ... is equivalent to ...
# SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
# Extract only the page_content attribute from the list of objects and put them back in a list
doc_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])

KeyboardInterrupt: ignored

Storing the embeddings in a vector database (Pinecone)

In [ ]:
pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import getpass    # To prompt the user for a password without echoing.
# from langchain.vectorstores import Pinecone

# PINECONE_API_KEY = getpass.getpass("Your API key: ")  # 2b975b00-9ceb-4d2f-b38d-ba6ccb6e532a
# PINECONE_ENV = getpass.getpass("Your env't name: ")   # us-west1-gcp-free

PINECONE_API_KEY = "2b975b00-9ceb-4d2f-b38d-ba6ccb6e532a"
PINECONE_ENV = "us-west1-gcp-free"

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV,  # next to api key in console
)

# all_indices = pinecone.list_indexes() # List all the indexed in our pinecone workspace
index_name = "discord-embeddings"
index_dimension = len(doc_embeddings[0])

# The number of embedded chunks
no_embeddings = len(chunks)
print('No of embeddings: ' + no_embeddings)   # The free pinecone API can't take more than 1000 vectors
print('Index dimension: ' + index_dimension)

384

In [ ]:
# Connect to the index
index = pinecone.Index(index_name)
# Current index statistics
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'first-upsert': {'vector_count': 1679}},
 'total_vector_count': 1679}

In [ ]:
#doc_embeddings

In [ ]:
step = 100  # This will be the size of the batch of vectors sent to pinecone at a time

for start in range(0, no_embeddings, step):
  # The end location of the current batch
  end = min(no_embeddings, start+step)    # If it reached the last batch, the end should be the total amount of vectors
                                          # [0..99], [100..199], ... , [1600..1678]  (The last batch should end at 1678)

  # create IDs for all embedded chunks (vectors) ... [0 -> 99 -> ... -> 1678]
  ids = [str(x) for x in range(start, end)]

  # create metadata for each vector ... (ideally, this should be as minimal as possible. For e.g., we can add the link to the median of the message chunk)
     # In this case, the original message is given as the metadata
  metadatas = [{'messages': chunk.page_content} for chunk in chunks[start:end]]

  # create a records list of current batch for upsert
  records = zip(ids, doc_embeddings[start:end], metadatas)

  # upsert to Pinecone
    # vectors = [ ( "id1", [0.1,0.2,..], {metadata1} )  ,  ( "id2", [0.4,0.6,..], {metadata2} )  , .. ]
    # namespace = "my-namespace"
  index.upsert(vectors=records, namespace="first-upsert")

  # index stat after current batch upsert
  print('Batch no. ' + str(int( start/step + 1 )) )
  index.describe_index_stats()

# index stats after all upsert batch
print('Completed upserting all batches: ')
index.describe_index_stats()

Batch no. 1
Batch no. 2
Batch no. 3
Batch no. 4
Batch no. 5
Batch no. 6
Batch no. 7
Batch no. 8
Batch no. 9
Batch no. 10
Batch no. 11
Batch no. 12
Batch no. 13
Batch no. 14
Batch no. 15
Batch no. 16
Batch no. 17
Completed upserting all batches: 


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'first-upsert': {'vector_count': 1679}},
 'total_vector_count': 1679}

In [ ]:

embedded_query = embeddings.embed_query("How do I open the PPP repo in a docker container")

query_response = index.query(
    namespace="first-upsert",
    top_k=10,
    include_values=False,
    include_metadata=True,
    vector=embedded_query
)

query_response

{'matches': [{'id': '144',
              'metadata': {'messages': 'Look in the PPP Docker container in '
                                       'the scripts folder for a script to '
                                       'query UTxOs.'},
              'score': 0.592360675,
              'values': []},
             {'id': '1651',
              'metadata': {'messages': 'Then a dialog asked if I wanted to '
                                       'open the project from within Docker. I '
                                       'said yes.'},
              'score': 0.543517053,
              'values': []},
             {'id': '1673',
              'metadata': {'messages': '<@665713765743853600> Could you '
                                       "explain what's happen with docker file "
                                       'in our repository after the'},
              'score': 0.541511595,
              'values': []},
             {'id': '1645',
              'metadata': {'messages': 'From t